references:

1- https://github.com/kunal-visoulia/Image-Restoration-using-SRCNN#:~:text=The%20SRCNN%20is%20a%20deep,resolution%20to%20high%20resolution%20images.&text=To%20evaluate%20the%20performance%20of,squared%20error%20(MSE)%2C%20and

2- https://github.com/Data-Science-Community-SRM/1080-Pixels

In [1]:
import os, shutil
from google.colab import drive
drive.mount('/content/drive')
!ls 'drive/MyDrive/Colab Notebooks/SRCNN'

Mounted at /content/drive
LR  Set14  Set5  source  SRCNN.ipynb  weights.h5


In [2]:
import sys
import keras
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage
import math
import os
import h5py

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.optimizers import Adam
from skimage.metrics import structural_similarity as ssim
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# python magic function, displays pyplot figures in the notebook instead of separate display window
%matplotlib inline

In [4]:
#define a function for peak signal-to-noise ratio (PSNR)
def psnr(target,ref):#target image and refernce image
    
    #assume RGB image and convert all integer values to float
    target_data=target.astype(float)
    ref_data=ref.astype(float)
    
    diff=ref_data-target_data
    diff=diff.flatten('C')#need ot flatten so computations can be done
    
    rmse=math.sqrt(np.mean(diff**2.))#2. for float values
    
    return 20*math.log10(255./rmse)

In [5]:
#define function for mean squared error(MSE)
def mse(target,ref):
    # the MSE between the two images is the sum of the squared difference between the two images
    err=np.sum((target.astype('float')-ref.astype('float'))**2)
    err=err/float(target.shape[0]*target.shape[1])#divided by total number of pixels
    
    return err

In [6]:
# define function that combines all three image quality metrics
def compare_images(target,ref):
    scores=[]
    scores.append(psnr(target,ref))
    scores.append(mse(target,ref))
    scores.append(ssim(target,ref,multichannel=True))#multichannel so that it can handle 3Dor 3 channel images RGB/BGR 
    
    return scores

In [7]:
import os
import cv2

# prepare degraded images by introducing quality distortions resizing

def prepare_images(path, factor):
    
    # loop through the files in the directory
    for file in os.listdir(path):
        
        # open the file
        img = cv2.imread(path + '/' + file)
        
        # find old and new image dimensions
        h, w, c = img.shape
        new_height = int(h / factor)
        new_width = int(w / factor)
        
        # resize the image - down
        img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_LINEAR)
        #interploation are methods for resizing images;how do you go from image with 100px to 1000px 
        #bilinear interpolation
        
        # resize the image - up
        img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)
        
        # save the image
        print('Saving {}'.format(file))
        cv2.imwrite(r'drive/MyDrive/Colab Notebooks/SRCNN/LR/{}'.format(file), img)

In [8]:
prepare_images(r'drive/MyDrive/Colab Notebooks/SRCNN/source/',3)
#source folder has high resolution images that will be converted to low resoltion images to be used for SRCNN

Saving t13.png
Saving t44.png
Saving t56.png
Saving tt24.png
Saving tt5.png
Saving t65.png
Saving t30.png
Saving tt22.png
Saving t66.png
Saving t52.png
Saving tt13.png
Saving tt18.png
Saving t47.png
Saving tt7.png
Saving tt16.png
Saving t63.png
Saving tt10.png
Saving t57.png
Saving t33.png
Saving t2.png
Saving t8.png
Saving t19.png
Saving t29.png
Saving t55.png
Saving t24.png
Saving t28.png
Saving t18.png
Saving t34.png
Saving t37.png
Saving t53.png
Saving t48.png
Saving t36.png
Saving t49.png
Saving t9.png
Saving t1.png
Saving t51.png
Saving tt23.png
Saving tt12.png
Saving t62.png
Saving t58.png
Saving t45.png
Saving t42.png
Saving t3.png
Saving t31.png
Saving t27.png
Saving t16.png
Saving t21.png
Saving tt9.png
Saving t23.png
Saving t15.png
Saving t4.png
Saving t32.png
Saving t43.png
Saving t20.png
Saving tt14.png
Saving t25.png
Saving t35.png
Saving tt8.png
Saving tt1.png
Saving tt4.png
Saving tt17.png
Saving t7.png
Saving t39.png
Saving t10.png
Saving tt2.png
Saving t6.png
Saving t

In [9]:
xtrain_path=r'drive/MyDrive/Colab Notebooks/SRCNN/LR'
ytrain_path=r'drive/MyDrive/Colab Notebooks/SRCNN/source'

In [10]:
low_res_path=os.listdir(xtrain_path)
high_res_path=os.listdir(ytrain_path)

In [11]:
low_res_path[0]

't13.png'

In [12]:
x_train_path=[]
y_train_path=[]
for i in range(0,len(low_res_path)):
    x_train_path.append(xtrain_path+"/"+low_res_path[i])
    y_train_path.append(ytrain_path+"/"+high_res_path[i])

In [13]:
x_train_path= sorted(x_train_path)
y_train_path=sorted(y_train_path)

In [14]:
Random_Crop = 30
Patch_size = 32
label_size = 20
conv_side = 6
scale = 2


def prepare_data(_path):
    nums = _path.__len__()

    data = np.zeros((nums * Random_Crop, 1, Patch_size, Patch_size), dtype=np.double)
    label = np.zeros((nums * Random_Crop, 1, label_size, label_size), dtype=np.double)

    for i in _path:
        name = i
        hr_img = cv2.imread(name, cv2.IMREAD_COLOR)
        shape = hr_img.shape

        hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
        hr_img = hr_img[:, :, 0]

        # two resize operation to produce training data and labels
        lr_img = cv2.resize(hr_img, (shape[1] / scale, shape[0] / scale))
        lr_img = cv2.resize(lr_img, (shape[1], shape[0]))

        # produce Random_Crop random coordinate to crop training img
        Points_x = np.random.randint(0, min(shape[0], shape[1]) - Patch_size, Random_Crop)
        Points_y = np.random.randint(0, min(shape[0], shape[1]) - Patch_size, Random_Crop)

        for j in range(Random_Crop):
            lr_patch = lr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]
            hr_patch = hr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]

            lr_patch = lr_patch.astype(float) / 255.
            hr_patch = hr_patch.astype(float) / 255.

            data[i * Random_Crop + j, 0, :, :] = lr_patch
            label[i * Random_Crop + j, 0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]
            # cv2.imshow("lr", lr_patch)
            # cv2.imshow("hr", hr_patch)
            # cv2.waitKey(0)
    return data, label

In [15]:
BLOCK_STEP = 16
BLOCK_SIZE = 32


def prepare_crop_data(_path):
    nums = _path.__len__()

    data = []
    label = []

    for i in _path:
        name = i
        hr_img = cv2.imread(name, cv2.IMREAD_COLOR)
        hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
        hr_img = hr_img[:, :, 0]
        shape = hr_img.shape

        # two resize operation to produce training data and labels
        lr_img = cv2.resize(hr_img, (shape[1] // scale, shape[0] // scale))
        lr_img = cv2.resize(lr_img, (shape[1], shape[0]))

        width_num = (shape[0] - (BLOCK_SIZE - BLOCK_STEP) * 2) // BLOCK_STEP
        height_num = (shape[1] - (BLOCK_SIZE - BLOCK_STEP) * 2) // BLOCK_STEP
        for k in range(width_num):
            for j in range(height_num):
                x = k * BLOCK_STEP
                y = j * BLOCK_STEP
                hr_patch = hr_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]
                lr_patch = lr_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]

                lr_patch = lr_patch.astype(float) / 255.
                hr_patch = hr_patch.astype(float) / 255.

                lr = np.zeros((1, Patch_size, Patch_size), dtype=np.double)
                hr = np.zeros((1, label_size, label_size), dtype=np.double)

                lr[0, :, :] = lr_patch
                hr[0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]

                data.append(lr)
                label.append(hr)

    data = np.array(data, dtype=float)
    label = np.array(label, dtype=float)
    return data, label

In [16]:
def write_hdf5(data, labels, output_filename):
    """
    This function is used to save image data and its label(s) to hdf5 file.
    output_file.h5,contain data and label
    """

    x = data.astype(np.float32)
    y = labels.astype(np.float32)

    with h5py.File(output_filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)
        # h.create_dataset()

In [17]:
def read_training_data(file):
    with h5py.File(file, 'r') as hf:
        data = np.array(hf.get('data'))
        label = np.array(hf.get('label'))
        train_data = np.transpose(data, (0, 2, 3, 1))
        train_label = np.transpose(label, (0, 2, 3, 1))
        return train_data, train_label

In [18]:
traindata, trainlabel = prepare_crop_data(y_train_path)

In [19]:
srcnn=Sequential()
srcnn.add(Conv2D(filters=128, kernel_size = (9, 9), kernel_initializer='glorot_uniform',
                 activation='relu', padding='valid', use_bias=True, input_shape=(None,None,1)))
srcnn.add(Conv2D(filters=64, kernel_size = (3, 3), kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', use_bias=True))
srcnn.add(Conv2D(filters=1, kernel_size = (5, 5), kernel_initializer='glorot_uniform',
                     activation='linear', padding='valid', use_bias=True))
adam = Adam(lr=0.0001)
srcnn.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
srcnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, None, None, 128)   10496     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    73792     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 1)     1601      
Total params: 85,889
Trainable params: 85,889
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [20]:
train_data = np.transpose(traindata, (0, 2, 3, 1))
train_label = np.transpose(trainlabel, (0, 2, 3, 1))
train_data.shape

(14901, 32, 32, 1)

In [21]:
model = srcnn.fit(train_data,train_label,batch_size=128,epochs=250,validation_split=0.2)
model.history

Epoch 1/250
94/94 [==============================] - 45s 21ms/step - loss: 0.0758 - mean_squared_error: 0.0758 - val_loss: 0.0022 - val_mean_squared_error: 0.0022
Epoch 2/250
94/94 [==============================] - 1s 13ms/step - loss: 0.0028 - mean_squared_error: 0.0028 - val_loss: 0.0012 - val_mean_squared_error: 0.0012
Epoch 3/250
94/94 [==============================] - 1s 14ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0010 - val_mean_squared_error: 0.0010
Epoch 4/250
94/94 [==============================] - 1s 14ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 9.2024e-04 - val_mean_squared_error: 9.2024e-04
Epoch 5/250
94/94 [==============================] - 1s 14ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 8.7823e-04 - val_mean_squared_error: 8.7823e-04
Epoch 6/250
94/94 [==============================] - 1s 14ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 8.5679e-04 - val_mean_squared_error: 8.5679e-04
Epo

{'loss': [0.0274825282394886,
  0.0024458137340843678,
  0.001799289952032268,
  0.0016161254607141018,
  0.0015292116440832615,
  0.0014743378851562738,
  0.001439752639271319,
  0.001409330521710217,
  0.001390294055454433,
  0.001373524428345263,
  0.0013609867310151458,
  0.001346917124465108,
  0.0013358367141336203,
  0.0013273798394948244,
  0.0013225094880908728,
  0.0013132730964571238,
  0.0013109876308590174,
  0.0013018412282690406,
  0.0012972673866897821,
  0.001294095884077251,
  0.0012871581129729748,
  0.0012844775337725878,
  0.0012814038200303912,
  0.0012737938668578863,
  0.0012719737133011222,
  0.00126792979426682,
  0.0012653353624045849,
  0.0012604399817064404,
  0.001257520285435021,
  0.0012530938256531954,
  0.0012542027980089188,
  0.0012471451191231608,
  0.0012498613214120269,
  0.0012429108610376716,
  0.0012386133894324303,
  0.0012397925602272153,
  0.001234017894603312,
  0.0012321914546191692,
  0.001236156327649951,
  0.001230068039149046,
  0.0012

In [22]:
final_model =srcnn.save('drive/MyDrive/Colab Notebooks/SRCNN/weights.h5')

In [23]:
# define necessary image processing functions

#necessary cuz when we run images through SRCNN based on the kernel sizes and convulational layers, we are going to lose some of these outside pixels
#the images are going to get smaller and that's why it is neccesary to have a divisible image size 
def modcrop(img,scale):
    #temp size
    tmpsz=img.shape
    sz=tmpsz[0:2]
    
    #ensures that dimension of our image are divisible by scale(doesn't leaves hanging remainders) by cropping the images size
    #np.mod returns the remainder bewtween our sz and scale
    sz=sz-np.mod(sz,scale)
    
    img=img[0:sz[0],1:sz[1]]
    return img

In [24]:
#crop offs the bordersize from all sides of the image
def shave(image,border):
    img=image[border: -border,border:-border]
    return img

In [41]:
#define main prediction function

def predict(image_path):
    
    #load the srcnn model with weights cuz deep learning neural n/w take lot of time to train(have to feed in large amount of input data)
    model = model()
    model.load_weights('drive/MyDrive/Colab Notebooks/SRCNN/weights.h5')
    model.fit()
     
    #load the degraded and reference images
    #in opencv, images are loaded as BGR channels
    path,file=os.path.split(image_path)
    degraded=cv2.imread(image_path)
    ref=cv2.imread(r'drive/MyDrive/Colab Notebooks/SRCNN/source/{}'.format(file))
    
    #preprocess the image with modcrop
    ref=modcrop(ref,3)
    #when calculating our image quality metrics later we have the same size image to what we produce in SRCNN network
    degraded=modcrop(degraded,3)
    
    #convert the image to YCrCb(3 channel image) - (srcnn trained on Y channel)
    temp=cv2.cvtColor(degraded,cv2.COLOR_BGR2YCrCb)
    #opencv does a very good job in converting from rgb to YCrCb and back
    
    #create image slice and normalize cuz SRCNN works on one dimensional input(or 3D inputs of depth 1 ,ie, inputs with one channel)
    Y=numpy.zeros((1,temp.shape[0],temp.shape[1],1),dtype=float)
    #create a numpy array the we fill with data,temp.shape[0]=width,[1]=height and last one means one channel(essentially like batch=1 cuz that's what going to get passed to the n/w ')
    #fill in the data; all values are normalized to between 0 and 1 as that's how srcnn was trained
    Y[0,:,:,0]=temp[:,:,0].astype(float)/255
    #first 0 means 0th index(we are saying that batch size is 1); :,: means every point in these channels; last 0 means first channel,ie, all the pixels in first luminescence channel
    #so we have our image slice, we have the Y channel, which is the first channel(index 0) out of the image that we converted to YCrCb color space
    
    #perform super-resolution with srcnn
    pre=srcnn.predict(Y,batch_size=1)#that's why we had index 0  above cuz we are saying that batch size is 1
    
    #post-process output cuz pre is still normalized
    pre*=255#multiplying every pixel by 255
    pre[pre[:]>255]=255#any pixels >255 set it =255 to prevent any rounding errors due to multiplication
    pre[pre[:]<0] =0# same reason as above
    pre=pre.astype(np.uint8)#convert float back to int values
    
    #cuz this is only the luminescence channel in the pre ,SO
    #copy Y channel back to image and convert to BGR
    temp=shave(temp,6)#accd.to tutor it loses 3 pixels on each side so if we shave this with a border 6,we can crop it appropriately there, so it is the same size as our output
    #if not agree with tutor, use print statements to see the specific dimensions
    
    # for the first channel(Y channel), copy in the output of our network
    temp[:,:,0]=pre[0,:,:,0]
    #So we are keeping the red difference and blue difference, channels 1 and 2, in this temp image which is in the YCrCb color space
    #and in the first one we are copying in our ouput,our luminiscence channel
    
    #convert back to bgr
    output=cv2.cvtColor(temp,cv2.COLOR_YCrCb2BGR)
    
    #emove borderfrom reference and degraded image, so that all our images(ref,degraded(low res.), and ouput(high res.))
    #are of the same size
    ref = shave(ref.astype(np.uint8), 6)
    degraded = shave(degraded.astype(np.uint8), 6)
    
    # image quality calculations
    scores = []
    scores.append(compare_images(degraded, ref))#degraded wrt ref
    scores.append(compare_images(output, ref))#high res. output wrt ref
    
    # return images and scores
    return ref, degraded, output, scores

In [42]:
for file in os.listdir('drive/MyDrive/Colab Notebooks/SRCNN/Set5'):
  # perform super-resolution
  ref, degraded, output, scores = predict('drive/MyDrive/Colab Notebooks/SRCNN/Set5')
 
  # display images as subplots
  fig, axs = plt.subplots(1, 3, figsize=(20, 8))
  axs[0].imshow(cv2.cvtColor(ref, cv2.COLOR_BGR2RGB))
  axs[0].set_title('Original')
  axs[1].imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
  axs[1].set_title('Degraded')
  axs[1].set(xlabel = 'PSNR: {}\nMSE: {} \nSSIM: {}'.format(scores[0][0], scores[0][1], scores[0][2]))
  axs[2].imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
  axs[2].set_title('SRCNN')
  axs[2].set(xlabel = 'PSNR: {} \nMSE: {} \nSSIM: {}'.format(scores[1][0], scores[1][1], scores[1][2]))
 
  # remove the x and y ticks
  for ax in axs:
      ax.set_xticks([])
      ax.set_yticks([])
print('Total PSNR: {}\nTotal MSE: {} \n'.format(scores[1][0]/5, scores[1][1]/5))
#print('Saving {}'.format(file))
#fig.savefig('output/{}.png'.format(os.path.splitext(file)[0])) 
#plt.close()

UnboundLocalError: ignored